In [10]:
'''
O-MINDS algorithm for sorting stimuli.

Takes in a matrix of information about each stimulus, and creates folders of 
stimuli based on the specified parameters.
'''
import numpy as np

import os
import sys
# install packages in virtual environment. If running in Non GUI mode, comment out the line below.
# os.system('python3 -m pip install pandas scipy --user --no-warn-script-location')
import math
import random
import shutil
import cmd
import pandas as pd
from scipy.spatial import distance

In [2]:
################################## STEP 0: ORGANIZE ###################################

#### user inputs overrides with arguments

current_dir = "/Users/alexgordienko/Documents/work/science/duncanlab/OMINDS/scripts/"
image_directory = '/Users/alexgordienko/Documents/work/science/duncanlab/OMINDS/scripts/objects'
num_folders = 2
num_stimuli = 3
memorability = 'n'
nameability = 'n'
emotionality = 'n'
orientationq = 'shoebox'
unique = 1
target_directory = '/Users/alexgordienko/Documents/work/science/duncanlab/OMINDS/results'
newname = 1

if(memorability != 'n'):
    memorability = int(memorability)
if(nameability != 'n'):
    nameability = int(nameability)
if(emotionality != 'n'):
    emotionality = int(emotionality)
newname = int(newname)
num_folders = int(num_folders)
num_stimuli = int(num_stimuli)
os.chdir(current_dir)
df = pd.read_csv("full_dt_pf.csv")
total_stim = num_folders * num_stimuli

def assert_exit(num_possible):
    try:
        assert total_stim <= num_possible
    except AssertionError:
        err_message = 'Too many stimuli requested given the parameters. Asked for ' + \
        str(total_stim) + ' but the maximum number of possible stimuli is ' + str(num_possible)  
        sys.exit(err_message)

folder_list = []

for i in range(num_folders):
    folder_list.append(pd.DataFrame())

print(folder_list)

[Empty DataFrame
Columns: []
Index: [], Empty DataFrame
Columns: []
Index: []]


In [3]:
#################################### STEP 1: DIFFERENCE SCORES ######################################
# find how far the score is compared to the ideal scores given

if memorability == 'n':
    if nameability == 'n':
        if emotionality == 'n':
            print('random scores')
            df['dist'] = df.apply(lambda row: random.random(), axis=1)
        else:
            print("just emotional")
            df['dist'] = df.apply(lambda row: 
                distance.euclidean((emotionality), [row['scaled_emotional']]),
                                  axis=1)
    else:
        if emotionality == 'n':
            print("just nameable")
            df['dist'] = df.apply(lambda row: 
                distance.euclidean((nameability), [row['scaled_name']]),
                                  axis=1)
        else:
            print("nameability and emotional")
            df['dist'] = df.apply(lambda row: 
                distance.euclidean((nameability, emotionality), 
                                   [row['scaled_name'],row['scaled_emotional']]),
                                  axis=1)
else:
    if nameability == 'n':
        if emotionality == 'n':
            print("just memorability")
            df['dist'] = df.apply(lambda row: 
                distance.euclidean((memorability), [row['scaled_memory_hits']]),
                                  axis=1)
        else:
            print("memorability and emotional")
            df['dist'] = df.apply(lambda row:
                                  distance.euclidean((memorability, emotionality), [row['scaled_memory_hits'], row['scaled_emotional']]),
                                  axis=1)
    else:
        if emotionality == 'n':
            print("memorability and nameability")
            df['dist'] = df.apply(lambda row: 
                distance.euclidean((memorability, nameability), [row['scaled_memory_hits'], row['scaled_name']]),
                                  axis=1)
        else:
            print("all 3")
            df['dist'] = df.apply(lambda row: 
                                  distance.euclidean((memorability, nameability, emotionality), 
                                                     [row['scaled_memory_hits'], row['scaled_name'], row['scaled_emotional']]),axis=1)

# sort by distance scores; should affect the rest of the script
df = df.sort_values(by='dist', ascending=True)
print(df["dist"])

random scores
138     0.000160
118     0.000204
671     0.000893
92      0.001433
1076    0.002808
          ...   
140     0.997993
516     0.998264
1150    0.998608
448     0.999714
342     0.999837
Name: dist, Length: 1748, dtype: float64


In [4]:
######################### STEP 2: UNIQUENESS TRIMMING ##############################

# if unique was selected, only keep the items with the best score for each modal name
if unique == "y":
    assert_exit(num_possible = 1174)
    # separate unique and non-unique values
    nonunique = df.loc[df['unique'] == 0]
    newtable = df.loc[df['unique'] == 1]

    # grab the best item for each unique modal response
    modalresponses = nonunique.modal_response.unique()
    for modalname in modalresponses:
        temp = nonunique.loc[nonunique['modal_response'] == modalname,].reset_index(drop=True)
        # temp = temp.sort_values(['dist'], ascending = [True])
        newtable = newtable.append(temp.iloc[0], ignore_index = True)

    df = newtable
else:
    assert_exit(num_possible = 1748)



In [5]:
############################### STEP 3: ORIENTATION Q SPLITTING #######################################

# If any of the orientation questions are selected, use the column to split the data.

if orientationq == "shoebox":
    lowtable = df.loc[df['shoebox_response'] == 'larger than a shoebox']
    hightable = df.loc[df['shoebox_response'] == 'smaller than a shoebox']
    assert_exit(len(hightable)*2)
    df_list = [lowtable, hightable]
elif orientationq == "humanmade":
    lowtable = df.loc[df['humanmade_response'] == 'natural']
    hightable = df.loc[df['humanmade_response'] == 'human-made']
    assert_exit(len(lowtable)*2)
    df_list = [lowtable, hightable]
elif orientationq == "outdoors":
    lowtable = df.loc[df['outdoors_response'] == 'indoors']
    hightable = df.loc[df['outdoors_response'] == 'outdoors']
    assert_exit(len(hightable)*2)
    df_list = [lowtable, hightable]
else:
    df_list = [df]

# if the items do not fit cleanly in the folders, store them
leftovers = pd.DataFrame()


In [6]:
for table in df_list:
    table = table.reset_index(drop=True)
    ############################# STEP 4: CUT TO PREFERRED SIZE ##################################

    # table = table.sort_values(['dist'], ascending=[True])
    cutoff = math.ceil(num_stimuli * num_folders/len(df_list))
    table = table.iloc[0:cutoff]
    print("cutoff", cutoff)
    print(len(table))

    ############################ STEP 5: FOLDER ALLOCATION ################################

    folder_assignment = [n for n in range(0, num_folders)]
    shuffled_list = []
    
    for group in range(math.ceil(num_stimuli/len(df_list))):
        shuffled_list += random.sample(folder_assignment, num_folders)
        print("SHUFFLED LIST", shuffled_list)
    print("FINAL SHUFFLED LIST", shuffled_list)
    print(folder_assignment, num_folders)
    print(shuffled_list)
    # if there's no leftovers after dividing into folders, then it's no problem
    
    print("range len table", range(len(table)))
    for i in range(len(table)):
        print("i", i)
        if (i < math.floor(len(table)/num_folders)*num_folders):
            stim_row = table.iloc[i]
#             print(stim_row)
            targ_folder = shuffled_list[i]
            print("targ folder", targ_folder)
            folder_list[targ_folder] = folder_list[targ_folder].append(stim_row)
        # put leftovers in a separate folder
        else:
            leftovers = leftovers.append(table.iloc[i])
print("shuffled list", shuffled_list)
print("folder_list", len(folder_list))
print("leftovers", leftovers)


cutoff 3
3
SHUFFLED LIST [0, 1]
SHUFFLED LIST [0, 1, 0, 1]
FINAL SHUFFLED LIST [0, 1, 0, 1]
[0, 1] 2
[0, 1, 0, 1]
range len table range(0, 3)
i 0
targ folder 0
i 1
targ folder 1
i 2
cutoff 3
3
SHUFFLED LIST [0, 1]
SHUFFLED LIST [0, 1, 1, 0]
FINAL SHUFFLED LIST [0, 1, 1, 0]
[0, 1] 2
[0, 1, 1, 0]
range len table range(0, 3)
i 0
targ folder 0
i 1
targ folder 1
i 2
shuffled list [0, 1, 1, 0]
folder_list 2
leftovers        stimulus  memorability  scaled_memory_hits  score_emotional  \
2  Wardrobe.jpg     -0.095738            0.161992        -0.472964   
2  Gun_Glue.jpg      0.032180            0.606754         0.160846   

   sd_emotional  score_humanmade  sd_humanmade  score_outdoors  sd_outdoors  \
2      0.507070              1.0           0.0        0.111111     0.333333   
2      0.603619              1.0           0.0        0.333333     0.516398   

   score_shoebox  ...  humanmade_response  outdoors_response  \
2            0.0  ...          human-made            indoors   
2       

In [7]:
###################### STEP 6: DOUBLE CHECK ###########################
# If there are an odd number of requested stimuli, but an even number of stimuli in leftovers, cut the worst-fitting item.

# print(leftovers)

if len(leftovers) != 0:
    leftovers = leftovers.sample(frac=1).reset_index(drop=True)
#     print(leftovers)
    for i in range(num_folders):
        folder_list[i] = folder_list[i].append(leftovers.iloc[i])
#         print("leftovers b", leftovers)
        
        
print(len(folder_list))

2


In [ ]:
#################### STEP 7: FOLDERS #####################

# create folders
# for i in range(num_folders):
#     folder_list[i] = folder_list[i].sample(frac=1)
#     folder = folder_list[i]
#     folder.index = range(len(folder.index))
#     foldernumber = str(i + 1) # begin indexing at 1

#     folderpath = target_directory + foldernumber + "/"

#     if not os.path.exists(folderpath):
#         os.makedirs(folderpath)

#     folder['newname'] = folder.apply(lambda _: '', axis=1)

#     for j, row in folder.iterrows():
        
#         stimname = row['stimulus']

#         # If they want it re-named, change to numbers beginning at index 1.
#         if newname == 1:
#             changedname = foldernumber + "_" + str(j + 1) + ".jpg" 
#         else:
#             changedname = foldernumber + "_" + stimname
        
#         # Record this in the output file in a new column
#         folder.loc[j,'newname'] = changedname

#         # Copy the image from the stored folder
#         itempath = image_directory + stimname
#         targpath = folderpath + changedname
#         # move it over
#         shutil.copy(itempath, targpath)


In [ ]:

#################### STEP 8: SUMMARIZED CSV ##########################

# outputCSV = pd.DataFrame()

# for folder in folder_list:
#     outputCSV = outputCSV.append(folder).reset_index(drop=True)

# outputCSV.to_csv(target_directory + "summary.csv", index = False, encoding='utf-8')

# sys.stdout.flush()

In [11]:
new = np.array([1, 2, 3, 4])
print(new)
new = new.sample(frac=1)
print(new)

[1 2 3 4]


AttributeError: 'numpy.ndarray' object has no attribute 'sample'